In [22]:
import pandas as pd
import jax.numpy as npx
import tensorflow as tf
from mlobject import *
from ceteris_paribus.explainer import explain
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn import metrics
from ceteris_paribus.profiles import individual_variable_profile
from ceteris_paribus.plots.plots import plot_notebook, plot

In [6]:
model = svm.SVC(gamma=0.001, C=100., probability=True)
    
model.fit(training_features, training_target)
certainty = metrics.accuracy_score(testing_target, model.predict(testing_features))
print("Accuracy:", certainty)

Accuracy: 0.7752808988764045


In [7]:
predict_fn = lambda x: model.predict_proba(x)

In [27]:
print(model.predict_proba(np.array([1, 0, 60, 50, 0, 0]).reshape(1, -1)))
print(testing_features[1])
print(np.array([1, 0, 60, 50, 0, 0]))

[[0.88069834 0.11930166]]
[ 3.   0.  21.  16.1  0.   0. ]
[ 1  0 60 50  0  0]


In [13]:
explainer_xgb = explain(model, variable_names=f, data=training_features, y=training_target, label='SVM', predict_function=lambda x: model.predict_proba(x)[::, 1])

In [20]:
cp_xgb = individual_variable_profile(explainer_xgb, np.array([1, 0, 60, 50, 0, 0]), np.array([0]))

In [24]:
plot(cp_xgb, selected_variables=["Sex"], width=700, height=800, size=4)

In [5]:
data = pd.read_csv('titanic.csv')
data.rename(columns={'Survived': 'class'}, inplace=True)
data['Sex'] = data['Sex'].map({'male':0,'female':1})
data['Embarked'] = data['Embarked'].map({'S':0,'C':1,'Q':2})
data['Relatives'] = data['SibSp'] + data['Parch']

data = data.drop(['PassengerId', 'Name','Ticket','Cabin', 'SibSp', 'Parch'], axis=1)
data = data.dropna()

f = ['Pclass', 'Sex', 'Age', 'Fare', 'Embarked', 'Relatives']

features = data.drop('class', axis=1).values
training_features, testing_features, training_target, testing_target = \
    train_test_split(features, data['class'].values, random_state=None)
